##### Copyright 2020 Los autores de TensorFlow.

# Diferenciación automática avanzada

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/advanced_autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar cuaderno</a>   </td>
</table>

La [guía de diferenciación automática](autodiff.ipynb) incluye todo lo necesario para calcular gradientes. Esta guía se centra en características más profundas y menos comunes de la API `tf.GradientTape` .

## Configuración

## Controlar la grabación de gradiente

En la [guía de diferenciación automática](autodiff.ipynb) , vio cómo controlar qué variables y tensores observa la cinta mientras realiza el cálculo del gradiente.

La cinta también tiene métodos para manipular la grabación.

Si desea detener la grabación de gradientes, puede usar `GradientTape.stop_recording()` para suspender temporalmente la grabación.

Esto puede resultar útil para reducir los gastos generales si no desea diferenciar una operación complicada en medio de su modelo. Esto podría incluir el cálculo de una métrica o un resultado intermedio:

Si desea empezar de nuevo por completo, utilice `reset()` . Simplemente salir del bloque de cinta de degradado y reiniciar suele ser más fácil de leer, pero puede usar `reset` cuando salir del bloque de cinta sea difícil o imposible.

## Detener gradiente

A diferencia de los controles de cinta globales anteriores, la función `tf.stop_gradient` es mucho más precisa. Se puede utilizar para evitar que los gradientes fluyan a lo largo de un camino particular, sin necesidad de acceder a la cinta misma:

## Degradados personalizados

En algunos casos, es posible que desees controlar exactamente cómo se calculan los gradientes en lugar de utilizar el valor predeterminado. Estas situaciones incluyen:

- No hay un gradiente definido para una nueva operación que estás escribiendo.
- Los cálculos predeterminados son numéricamente inestables.
- Desea almacenar en caché un cálculo costoso del pase directo.
- Quiere modificar un valor (por ejemplo usando: `tf.clip_by_value` , `tf.math.round` ) sin modificar el degradado.

Para escribir una nueva operación, puede usar `tf.RegisterGradient` para configurar la suya propia. Consulte esa página para obtener más detalles. (Tenga en cuenta que el registro de gradiente es global, así que cámbielo con precaución).

Para los últimos tres casos, puede utilizar `tf.custom_gradient` .


Aquí hay un ejemplo que aplica `tf.clip_by_norm` al gradiente intermedio.

Consulte el decorador `tf.custom_gradient` para obtener más detalles.

## Varias cintas

Varias cintas interactúan sin problemas. Por ejemplo, aquí cada cinta observa un conjunto diferente de tensores:

### gradientes de orden superior

Las operaciones dentro del administrador de contexto `GradientTape` se registran para una diferenciación automática. Si los gradientes se calculan en ese contexto, entonces el cálculo del gradiente también se registra. Como resultado, exactamente la misma API también funciona para gradientes de orden superior. Por ejemplo:

Si bien eso proporciona la segunda derivada de una función *escalar* , este patrón no se generaliza para producir una matriz de Hesse, ya que `GradientTape.gradient` solo calcula el gradiente de un escalar. Para construir un hessiano, consulte el [ejemplo de hesse](#hessian) en la [sección jacobiana](#jacobians) .

Las "llamadas anidadas a `GradientTape.gradient` " son un buen patrón cuando se calcula un escalar a partir de un gradiente y luego el escalar resultante actúa como fuente para un segundo cálculo de gradiente, como en el siguiente ejemplo.


#### Ejemplo: regularización del gradiente de entrada

Muchos modelos son susceptibles a "ejemplos contradictorios". Esta colección de técnicas modifica la entrada del modelo para confundir la salida del modelo. La [implementación más simple](https://www.tensorflow.org/tutorials/generative/adversarial_fgsm) toma un solo paso a lo largo del gradiente de la salida con respecto a la entrada; el "gradiente de entrada".

Una técnica para aumentar la robustez ante ejemplos contradictorios es [la regularización del gradiente de entrada](https://arxiv.org/abs/1905.11468) , que intenta minimizar la magnitud del gradiente de entrada. Si el gradiente de entrada es pequeño, entonces el cambio en la salida también debería ser pequeño.

A continuación se muestra una implementación ingenua de la regularización del gradiente de entrada. La implementación es:

1. Calcule el gradiente de la salida con respecto a la entrada utilizando una cinta interior.
2. Calcule la magnitud de ese gradiente de entrada.
3. Calcule el gradiente de esa magnitud con respecto al modelo.

## jacobianos


Todos los ejemplos anteriores tomaron los gradientes de un objetivo escalar con respecto a algunos tensores de origen.

La [matriz jacobiana](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant) representa los gradientes de una función con valores vectoriales. Cada fila contiene el gradiente de uno de los elementos del vector.

El método `GradientTape.jacobian` le permite calcular de manera eficiente una matriz jacobiana.

Tenga en cuenta que:

- Como `gradient` : el argumento `sources` puede ser un tensor o un contenedor de tensores.
- A diferencia `gradient` : el tensor `target` debe ser un tensor único.

### fuente escalar

Como primer ejemplo, aquí está el jacobiano de un vector objetivo con respecto a una fuente escalar.

Cuando tomas el jacobiano con respecto a un escalar, el resultado tiene la forma del **objetivo** y proporciona el gradiente de cada elemento con respecto a la fuente:

### fuente tensorial

Ya sea que la entrada sea escalar o tensorial, `GradientTape.jacobian` calcula de manera eficiente el gradiente de cada elemento de la fuente con respecto a cada elemento de los objetivos.

Por ejemplo, la salida de esta capa tiene la forma `(10, 7)` :

Y la forma del núcleo de la capa es `(5, 10)` :

La forma del jacobiano de la salida con respecto al núcleo son esas dos formas concatenadas entre sí:

Si suma las dimensiones del objetivo, le quedará el gradiente de la suma que habría sido calculado por `GradientTape.gradient` :

<a id="hessian"> </a>

#### Ejemplo: arpillera

Si bien `tf.GradientTape` no proporciona un método explícito para construir una matriz de Hesse, es posible construir una usando el método `GradientTape.jacobian` .

Nota: La matriz de Hesse contiene `N**2` parámetros. Por este y otros motivos no resulta práctico para la mayoría de modelos. Este ejemplo se incluye más como una demostración de cómo utilizar el método `GradientTape.jacobian` y no es un respaldo a la optimización directa basada en Hesse. Un producto vectorial de Hesse se puede [calcular de manera eficiente con cintas anidadas](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/eager/benchmarks/resnet50/hvp_test.py) y es un enfoque mucho más eficiente para la optimización de segundo orden.


Para usar este Hessiano para un paso del método de Newton, primero debes aplanar sus ejes en una matriz y aplanar el gradiente en un vector:

La matriz de Hesse debe ser simétrica:

El paso de actualización del método de Newton se muestra a continuación.

Nota: [En realidad, no invierta la matriz](https://www.johndcook.com/blog/2010/01/19/dont-invert-that-matrix/) .

Si bien esto es relativamente simple para un solo `tf.Variable` , aplicarlo a un modelo no trivial requeriría una concatenación y un corte cuidadosos para producir un Hessian completo en múltiples variables.

### Lote jacobiano

En algunos casos, desea tomar el jacobiano de cada uno de una pila de objetivos con respecto a una pila de fuentes, donde los jacobianos para cada par objetivo-fuente son independientes.

Por ejemplo, aquí la entrada `x` tiene forma `(batch, ins)` y la salida `y` tiene forma `(batch, outs)` .


El jacobiano completo de `y` con respecto a `x` tiene una forma de `(batch, ins, batch, outs)` , incluso si solo quieres `(batch, ins, outs)` .

Si los gradientes de cada elemento de la pila son independientes, entonces cada `(batch, batch)` de este tensor es una matriz diagonal:

Para obtener el resultado deseado, puede sumar la dimensión `batch` duplicado o seleccionar las diagonales usando `tf.einsum` .


Sería mucho más eficiente hacer el cálculo sin la dimensión adicional en primer lugar. El método `GradientTape.batch_jacobian` hace exactamente eso.

Precaución: `GradientTape.batch_jacobian` solo verifica que la primera dimensión del origen y el destino coincidan. No comprueba que los gradientes sean realmente independientes. Depende del usuario asegurarse de utilizar únicamente `batch_jacobian` cuando tenga sentido. Por ejemplo, agregar `layers.BatchNormalization` destruye la independencia, ya que se normaliza en toda la dimensión `batch` :

En este caso `batch_jacobian` aún se ejecuta y devuelve *algo* con la forma esperada, pero su contenido no tiene un significado claro.